# Chapter 4: Text and Bytes

Humans use text. Computers speak bytes.

## 1. Character Issues

a string is a sequence of characters. 那么问题来了：什么是character?

Unicode character:
- U+20AC (euro sign): U+4～6个16进制数
- About 10% of the valid code points have characters assigned to them
- unicode str 可以被看作"human text"

encoding:
- an algorithm that converts code points to byte sequences and vice versa.
- we **encode** str to bytes for storage or transmission
- we **decode** bytes to str to get human-readable text

In [1]:
s = 'café'
len(s) # 4 unicode char

4

In [2]:
b = s.encode('utf8')  # Encoder using UTF-8
b

b'caf\xc3\xa9'

In [3]:
len(b)  # é 在unicode 中是1个character，在utf-8 中是两个characters，因此长度为5.

5

In [4]:
b.decode('utf8')  # decode utf-8 bytes to str using UTF-8

'café'

## 2. Byte Essentials

Each item in bytes or bytearray is an integer from 0 to 255

In [8]:
cafe = bytes('café', encoding='utf_8')  # 通过str 构建byte，需要指明encoding
cafe

b'caf\xc3\xa9'

In [10]:
len(cafe)  # 5个items

5

In [11]:
for c in cafe:  # 每个item 是一个0～255 的数字
    print(c)

99
97
102
195
169


In [20]:
cafe[0]  # retrieves an int

b'c'

In [21]:
cafe[:1]  # Slices of bytes are also bytes — even slices of a single byte.

99

In [22]:
cafe[:1] == cafe[:1]  # 相同

True

In [18]:
cafe_arr = bytearray(cafe)
cafe_arr

bytearray(b'caf\xc3\xa9')

In [19]:
bytearray(b'caf\xc3\xa9')  # bytearray 的slice 还是bytearray
cafe_arr[-1:]

bytearray(b'\xa9')

Bytes 支持str 绝大多数的方法：
- casefold, isdecimal, isidentifier, isnumeric, isprintable
- endswith, replace, strip, translate, upper,

bytes 多一个str 不支持的方法：
- fromhex

In [23]:
bytes.fromhex('31 4B CE A9')

b'1K\xce\xa9'

### 2.1 Structs and Memory Views

## 3. Basic Encoders / Decoders

## 4. Understanding Encoder / Decoder Problems

## 5. handling Text Files

## 6. Normalizing Unicode for Saner Comparisons

## 7. Sorting Unicode Text

## 8. The Unicode Database

## 9. Dual-Mode str and bytes APIs

## 10. Chapter Summary